In [8]:
import numpy as np
from scipy import optimize, interpolate
import pandas as pd
from collections import namedtuple
import os
import shutil
import rwforcReader, rbdoutReader

In [9]:
def read_test_results(fileName):
    """
    ToDO
    """
    tests_dict = {}
    excel = pd.ExcelFile(fileName)
    for sheet in excel.sheet_names:
        df = excel.parse(sheet)
        value = df.values
        #print(sheet)
        #print(value.shape)
        tests_dict[sheet] = value
    return tests_dict
    


In [10]:
test = read_test_results('Z.xlsx')

In [11]:
def make_curve(A, n):
    """
    TODO
    """
    lt_zero = np.arange(-1, 0, 0.1)
    lt_zero_stress = -0.2 * lt_zero
    gt_zero = np.arange(0, 1, 0.01)
    gt_zero_stress = A * np.power(gt_zero, n)
    x = np.hstack([lt_zero, gt_zero])
    y = np.hstack([lt_zero_stress, gt_zero_stress])
    return np.hstack([x.reshape(-1,1), y.reshape(-1,1)])

In [12]:
def write_curve(fileName, curve, lcid):
    """
    TODO
    """
    with open(fileName, 'w+') as f:
        f.write("*KEYWORD\n")
        f.write("*DEFINE_CURVE\n")
        f.write("{0}".format(str(int(lcid)))+"\n")
        f.write("$#\n")
        
        for p in curve:
            f.write("{:.4f}, {:.4f}\n".format(p[0], p[1]))

        f.write("*END")


In [13]:
def prepare_files(folder_name, files):
    """
    TODO
    """
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    for f in files:
        shutil.copy(os.path.join(owd,f), os.path.join(owd, folder_name, f))#os.path.join(owd,folder_name,f), os.path.join(owd,f))
    os.chdir(folder_name)
    

In [14]:
def execute_calculation(exe_path, main_file):
    os.system(exe_path + " i=" + main_file + " NCPU=8 Memory=2000m")
    os.chdir

In [15]:
def get_result(rgb_id, rw_id):
    rwf = rwforcReader.rwforcReader('JellyRoll_Z.rwforc')
    #print(res[0]["1"])# 
    rbd = rbdoutReader.rbdoutReader('JellyRoll_Z.rbdout')
    # print(res[0]['time'], res[1]['2'])
    d_x = np.array(rbd[1][str(rgb_id)]).reshape(-1, 1)
    forc = np.array(rwf[1][str(rw_id)]).reshape(-1, 1)
    assert d_x.shape[0] == forc.shape[0]
    return np.hstack([d_x, forc])

In [16]:
def func_simulation(param):
    """
    TODO
    """
    global cur_num
    with open ("param.txt", "a+") as f:
        f.write("{0:d},{1:.4f},{2:.4f}\n".format(cur_num, param[0], param[1]))
    curve = make_curve(param[0], param[1])
    prepare_files(str(cur_num), ['JellyRoll.blk', 'JellyRoll.mat', 'JellyRoll_punch.dyn'])
    write_curve('2100.k', curve, 2100)
    #execute_calculation('D:\LSDYNA\program\ls-dyna_smp_s_R11_0_winx64_ifort131.exe', 'JellyRoll_punch.dyn')
    simu = get_result(2, 3)
    print(simu)
    os.chdir(owd)
    cur_num += 1
    return simu

def err_func(param, test):
    print(test)
    simu = func_simulation(param)
    funcSim = interpolate.interp1d(simu[:,0], simu[:,1], bounds_error=False)
    funcTest = interpolate.interp1d(test['Y'][:,0], test['Y'][:,1], bounds_error=False)
    if np.max(test['Y'][:,0]) > np.max(simu[:,0]):
        top = np.max(simu[:,0])
    else:
        top = np.max(test['Y'][:,0])
    #print(top)
    if np.min(test['Y'][:,0]) > np.min(simu[:,0]):
        lower = np.min(test["Y"][:,0])
    else:
        lower = np.min(simu[:,0])
    
    inter_sim = funcSim(np.arange(lower,top, 0.001))
    inter_test = funcTest(np.arange(lower,top, 0.001))
    print(inter_sim)
    return inter_sim - inter_test


def do_inverse(test):
    """
    TODO
    """
    try:
        res = optimize.leastsq(err_func, (300, 1.5), args=[test], epsfcn=0.01)
        print(res)
    except ValueError as e:
        print(res)
    finally:
        os.chdir(owd)
    return res



In [17]:
cur_num = 1
owd = os.getcwd()
err_func((200,1), test)

{'Y': array([[1.720000e-03, 7.290000e+01],
       [3.430000e-03, 7.190000e+01],
       [6.030000e-03, 7.190000e+01],
       ...,
       [3.718646e+01, 2.375340e+04],
       [3.718783e+01, 2.375340e+04],
       [3.718987e+01, 2.375450e+04]])}


FileNotFoundError: [Errno 2] No such file or directory: 'JellyRoll_Z.rwforc'